In [1]:
# Standard imports for manipulating and visualising data
import pandas as pd
import numpy as np
import seaborn as sns
import sys
import numpy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from copy import deepcopy
from collections import Counter
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors 
from sklearn.neighbors import KNeighborsClassifier
from math import isclose

In [116]:
path = 'http://archive.ics.uci.edu/ml/machine-learning-databases/image/segmentation.data'
df = pd.read_table(path)
df.shape

(212, 1)

In [21]:
# Split the classes
majority_class = deepcopy(df)
minority_class = deepcopy(df)

minority_class = minority_class[(minority_class.target ==1)]
majority_class = majority_class[(majority_class.target ==0)]

# Clean up index
minority_class = minority_class.reset_index()
del minority_class['index']
majority_class = majority_class.reset_index()
del majority_class['index']

X_train = deepcopy(minority_class)
X_test = deepcopy(majority_class)
y_train = X_train.target
y_test = X_test.target
del X_train["target"]
del X_test["target"]

# Scale Data
X_train = pd.get_dummies(X_train)
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = pd.get_dummies(X_test)
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
scaler.fit(X_test)
X_test = scaler.transform(X_test)

# Fit the model and find the counterfactual-set cf(x,p) using KNeighborsClassifier
# the CF-Set pair instances either side of decision
# It pairs an instance in the majority space and its counterfactually-related instance in the minority space
knn = KNeighborsClassifier(n_neighbors=1, algorithm='auto', metric='euclidean') 
knn.fit(X_train, y_train)
for i in range(len(X_test)):
    instance = X_test[i]
    indxs = knn.kneighbors(X_test, return_distance=False)
    break
rows = indxs[0: , :]
dic = pd.DataFrame(rows)
list_of_counterfactual = dic[0].to_list()
counterfactual_instances = minority_class.iloc[list_of_counterfactual]
factual_instances = deepcopy(majority_class)

# Clean up index
counterfactual_instances = counterfactual_instances.reset_index()
del counterfactual_instances['index']
majority_class = majority_class.reset_index()
del majority_class['index']

# Select the number of synthetic esmaples to be generated
for idx, row in df.iterrows():
    num_of_maj = df[df.target == 0].shape[0]
    num_of_min = df[df.target == 1].shape[0]
    num_syn = num_of_maj - num_of_min  
    factual_instances = factual_instances.head(n=num_syn)
    counterfactual_instances = counterfactual_instances.head(n=num_syn)

# Define the Tolerance and the Feature Differences
# Define the mean and standard deviation for each feature
for column in df:
    mean_values = df.apply(np.mean).tolist()
    std_values = df.apply(np.std).tolist()
    mean_column = pd.DataFrame(mean_values, columns=['Mean'])
    std_column = pd.DataFrame(std_values, columns=['STD'])
    mean_std = pd.concat([mean_column,std_column], axis=1)

# Calculate the tolerance
mean_std["std_value"] = 0
mean_std["tolerance"] = 0
for ind, row in mean_std.iterrows():
    mean_std.loc[ind, "std_value"] = row['Mean'] + (0.2*row['STD'])
for ind, row in mean_std.iterrows():
    mean_std.loc[ind, "tolerance"] = row['std_value'] - (row['Mean'])

# Calculate the feature differences
atol_list = mean_std['tolerance'].values.tolist()
def compare_isclose(factual_instances, counterfactual_instances, atol_list):
    feature_list = []
    for i, col in enumerate(factual_instances.columns):
        feature_list.append(np.isclose(factual_instances[col], counterfactual_instances[col], atol=atol_list[i]))
        feature_df = pd.DataFrame.from_records(feature_list).T
        return feature_df
feature_df = compare_isclose(factual_instances, counterfactual_instances, atol_list)
feature_df = feature_df.iloc[:, :-1]
num_of_col= len(feature_df.columns)

feature_df['similar'] = feature_df.sum(axis=1)

for idx, row in feature_df.iterrows():
    try:
        similar = feature_df.loc[idx, 'similar']
        feature_df.at[idx, "differences"] = (num_of_col - similar)
    except:
        pass
    
# Applying the feature differences to the dataset and select the reuqired number of differences
for idx, row in factual_instances.iterrows():
    try:
        differences = feature_df.loc[idx, 'differences']
        factual_instances.at[idx, "differences"] = differences
    except:
        pass

for idx, row in counterfactual_instances.iterrows():
    try:
        differences = feature_df.loc[idx, 'differences']
        counterfactual_instances.at[idx, "differences"] = differences
    except:
        pass

factual_instances = factual_instances[(factual_instances.differences <=2) ]
counterfactual_instances = counterfactual_instances[(counterfactual_instances.differences <=2)]

del factual_instances['differences']
del counterfactual_instances['differences']

#  Define unpaired instances (x') by filtering out majority instances in CF-Set usinv merge function
all_cases = deepcopy(df)
all_cases = all_cases[(all_cases.target ==0)]
merged = all_cases.merge(factual_instances, how='left', indicator=True)
unique_x_prime = merged[merged['_merge']=='left_only']
del unique_x_prime['_merge']
unique_x_prime.shape

# For each paired instance 𝒙, from the majority class, find its nearest-neighbor unpaired instance x'
X_train = deepcopy(unique_x_prime)
X_test = deepcopy(factual_instances)
y_train = X_train.target
y_test = X_test.target
del X_train["target"]
del X_test["target"]

# Scale Data
X_train = pd.get_dummies(X_train)
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = pd.get_dummies(X_test)
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
scaler.fit(X_test)
X_test = scaler.transform(X_test)

# Fit model to find x' for each x
knn = KNeighborsClassifier(n_neighbors=1, algorithm='auto', metric='euclidean') 
knn.fit(X_train, y_train)  
for i in range(len(X_test)):
    instance = X_test[i]
    indxs = knn.kneighbors(X_test, return_distance=False)
    break
rows = indxs[0: , :]
dic = pd.DataFrame(rows)
list_of_names = dic[0].to_list()

x_prime = unique_x_prime.iloc[list_of_names]

# Create New Data Points (1st step: factual & counterfactual)
del factual_instances['target']
del counterfactual_instances['target']

get_tolerance = pd.DataFrame(atol_list, 
             columns=['digits'])
get_tolerance = get_tolerance.iloc[:-1, :]

# Transfer feature values from p to p' using the tolerance of 0.5  
p_prime = counterfactual_instances.where(counterfactual_instances.sub(factual_instances).abs().gt(get_tolerance), )

# Clean up index
x_prime = x_prime.reset_index()
del x_prime['index']
# Clean up index
p_prime = p_prime.reset_index()
del p_prime['index']
del x_prime['target']

# Transfer remaining feature values from x' to p' 
p_prime[p_prime.isnull()] = x_prime
p_prime['target'] = 1

new_train = pd.concat([df, p_prime])

In [22]:
# Checking the frequency of the target feature
new_train.target.value_counts()

0    681
1    681
Name: target, dtype: int64

In [39]:
df.duplicated().sum()

703

In [48]:
df = pd.read_csv("/Users/mohammed.temraz.90/Desktop/PHD/PhD Projects/4th project/new_dataset_diff_ratio/winequality-red/4-5/wine5-4.csv")
df.target.value_counts()

0    681
1     53
Name: target, dtype: int64

In [113]:
df = pd.read_csv("/Users/mohammed.temraz.90/Desktop/PHD/PhD Projects/4th project/new_dataset_diff_ratio/winequality-red/4-5/wine5-4.csv")

# Split the classes
majority_class = deepcopy(df)
minority_class = deepcopy(df)
minority_class = minority_class[(minority_class.target ==1)]
majority_class = majority_class[(majority_class.target ==0)]

# Clean up index
minority_class = minority_class.reset_index()
del minority_class['index']
majority_class = majority_class.reset_index()
del majority_class['index']

X_train = deepcopy(minority_class)
X_test = deepcopy(majority_class)
y_train = X_train.target
y_test = X_test.target
del X_train["target"]
del X_test["target"]

# Scale Data
X_train = pd.get_dummies(X_train)
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = pd.get_dummies(X_test)
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
scaler.fit(X_test)
X_test = scaler.transform(X_test)

# Fit the model and find the counterfactual-set cf(x,p) using KNeighborsClassifier
# the CF-Set pair instances either side of decision
# It pairs an instance in the majority space and its counterfactually-related instance in the minority space
knn = KNeighborsClassifier(n_neighbors=1, algorithm='auto', metric='euclidean') 
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)    
for i in range(len(X_test)):
    instance = X_test[i]
    indxs = knn.kneighbors(X_test, return_distance=False)
    break
rows = indxs[0: , :]
dic = pd.DataFrame(rows)
list_of_names = dic[0].to_list()
counterfactual_instances = minority_class.iloc[list_of_names]
factual_instances = deepcopy(majority_class)

# Clean up index
counterfactual_instances = counterfactual_instances.reset_index()
del counterfactual_instances['index']
# Clean up index
majority_class = majority_class.reset_index()
del majority_class['index']

#  Select the number of synthetic esmaples to be generated
for idx, row in df.iterrows():
    num_of_maj = df[df.target == 0].shape[0]
    num_of_min = df[df.target == 1].shape[0]
    num_syn = num_of_maj - num_of_min  
factual_instances = factual_instances.head(n=num_syn)
counterfactual_instances = counterfactual_instances.head(n=num_syn)

# Define the Tolerance and the Feature Differences
# Define the mean and standard deviation for each feature
for column in df:
    mean_values = df.apply(np.mean).tolist()
    std_values = df.apply(np.std).tolist()
    mean_column = pd.DataFrame(mean_values, columns=['Mean'])
    std_column = pd.DataFrame(std_values, columns=['STD'])
    mean_std = pd.concat([mean_column,std_column], axis=1)

# Calculate the tolerance
mean_std["std_value"] = 0
mean_std["tolerance"] = 0
for ind, row in mean_std.iterrows():
    mean_std.loc[ind, "std_value"] = row['Mean'] + (1*row['STD'])
for ind, row in mean_std.iterrows():
    mean_std.loc[ind, "tolerance"] = row['std_value'] - (row['Mean'])
    
# Calculate the feature differences
atol_list = mean_std['tolerance'].values.tolist()
def compare_isclose(factual_instances, counterfactual_instances, atol_list):
    feature_list = []
    for i, col in enumerate(factual_instances.columns):
        feature_list.append(np.isclose(factual_instances[col], counterfactual_instances[col], atol=atol_list[i]))
    feature_df = pd.DataFrame.from_records(feature_list).T
    return feature_df
feature_df = compare_isclose(factual_instances, counterfactual_instances, atol_list)
feature_df = feature_df.iloc[:, :-1]
num_of_col= len(feature_df.columns)
feature_df['similar'] = feature_df.sum(axis=1)

for idx, row in feature_df.iterrows():
    try:
        similar = feature_df.loc[idx, 'similar']
        feature_df.at[idx, "differences"] = (num_of_col - similar)
    except:
        pass
    
# Applying the feature differences to the dataset and select the reuqired number of differences
for idx, row in factual_instances.iterrows():
    try:
        differences = feature_df.loc[idx, 'differences']
        factual_instances.at[idx, "differences"] = differences
    except:
        pass
for idx, row in counterfactual_instances.iterrows():
    try:
        differences = feature_df.loc[idx, 'differences']
        counterfactual_instances.at[idx, "differences"] = differences
    except:
        pass

factual_instances = factual_instances[(factual_instances.differences <=2) ]
counterfactual_instances = counterfactual_instances[(counterfactual_instances.differences <=2)]
del factual_instances['differences']
del counterfactual_instances['differences']

# Define unpaired instances (x') by filtering out majority instances in CF-Set usinv merge function
all_cases = deepcopy(df)
all_cases = all_cases[(all_cases.target ==0)]
merged = all_cases.merge(factual_instances, how='left', indicator=True)
unique_x_prime = merged[merged['_merge']=='left_only']
del unique_x_prime['_merge']
unique_x_prime.shape

# For each paired instance 𝒙, from the majority class, find its nearest-neighbor unpaired instance x'
X_train = deepcopy(unique_x_prime)
X_test = deepcopy(factual_instances)
y_train = X_train.target
y_test = X_test.target
del X_train["target"]
del X_test["target"]

# Scale Data
X_train = pd.get_dummies(X_train)
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = pd.get_dummies(X_test)
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
scaler.fit(X_test)
X_test = scaler.transform(X_test)

# Fit model to find x' for each x
knn = KNeighborsClassifier(n_neighbors=1, algorithm='auto', metric='euclidean') 
knn.fit(X_train, y_train)
for i in range(len(X_test)):
    instance = X_test[i]
    indxs = knn.kneighbors(X_test, return_distance=False)
    break
rows = indxs[0: , :]
dic = pd.DataFrame(rows)
list_of_names = dic[0].to_list()

x_prime = unique_x_prime.iloc[list_of_names]

# Create New Data Points
del factual_instances['target']
del counterfactual_instances['target']
get_tolerance = pd.DataFrame(atol_list, 
             columns=['digits'])
get_tolerance = get_tolerance.iloc[:-1, :]

# Transfer feature values from p to p_prime
p_prime = counterfactual_instances.where(counterfactual_instances.sub(factual_instances).abs().gt(get_tolerance), )

# Clean up index
x_prime = x_prime.reset_index()
del x_prime['index']

# Clean up index
p_prime = p_prime.reset_index()
del p_prime['index']
del x_prime['target']

# Transfer remaining feature values from x' to p' 
p_prime[p_prime.isnull()] = x_prime
p_prime['target'] = 1
new_train = pd.concat([df, p_prime])

In [114]:
# Checking the frequency of the target feature
new_train.target.value_counts()

0    681
1    522
Name: target, dtype: int64

In [ ]:
unique_p = unique_x_prime
p_instances = x_prime